For time series forecasting we're leveraging AutoGluon Framework since it provides support for a 
large number of probabilistic algorithms including deep learning and transformer based algorithms.

This notebook contains the code for the following steps:
1. Model selection
2. Hyperparameter tuning 

The model selected in this step is used for Model training and deployment in the next steps.

In [ ]:
import pandas as pd

In [3]:
!pwd

/home/sagemaker-user/Homepro_AWS_competancy/notebooks


### Reading data saved from Feature creation

#### Read timeseries and static features data

In [4]:
## read the file 
df_ts = pd.read_csv('../data/final_aggregated_data.csv')
df_ts.rename(columns={'transaction_date':'timestamp','sales_quantity_sum':'target'},inplace = True)
df_ts

,item_id,timestamp,target,net_revenue_sum,listing_price_mean,net_unit_price,promo_flag,holiday_flag,weekend_flag,holiday_ahead,promo_ahead,item_height,item_width,item_length,item_weight,article_category
0,ITEM0001,2020-01-01,640.0,35200.00,55.000000,55.000000,True,True,False,False,True,29.53,82.49,89.19,4.01,Electronics
1,ITEM0001,2020-01-02,838.0,46090.00,55.000000,55.000000,True,False,False,False,True,29.53,82.49,89.19,4.01,Electronics
2,ITEM0001,2020-01-03,956.0,52580.00,55.000000,55.000000,True,False,False,False,True,29.53,82.49,89.19,4.01,Electronics
3,ITEM0001,2020-01-04,1467.0,80685.00,55.000000,55.000000,True,False,True,False,True,29.53,82.49,89.19,4.01,Electronics
4,ITEM0001,2020-01-05,1372.0,75456.47,55.007605,54.997427,True,False,True,False,True,29.53,82.49,89.19,4.01,Electronics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122842,ITEM0100,2024-01-27,42.0,56700.00,1350.000000,1350.000000,True,False,True,False,False,92.30,16.26,81.72,17.57,Electronics
122843,ITEM0100,2024-01-28,55.0,74250.00,1350.000000,1350.000000,False,False,True,False,False,92.30,16.26,81.72,17.57,Electronics
122844,ITEM0100,2024-01-29,46.0,62100.00,1350.000000,1350.000000,False,False,False,False,False,92.30,16.26,81.72,17.57,Electronics
122845,ITEM0100,2024-01-30,32.0,43200.00,1350.000000,1350.000000,False,False,False,False,False,92.30,16.26,81.72,17.57,Electronics


In [5]:
df_static = pd.read_csv('../data/article_metadata.csv')
df_static

,item_id,item_height,item_width,item_length,item_weight,article_category
0,ITEM0001,29.53,82.49,89.19,4.01,Electronics
1,ITEM0002,69.36,2.95,19.64,18.04,Toys
2,ITEM0003,86.38,59.71,34.78,6.12,Home
3,ITEM0004,83.84,78.03,89.58,15.34,Clothing
4,ITEM0005,16.68,44.55,64.05,17.96,Electronics
...,...,...,...,...,...,...
495,ITEM0496,50.30,63.68,84.41,8.84,Sports
496,ITEM0497,46.07,78.31,48.13,5.50,Home
497,ITEM0498,24.89,19.40,77.16,0.74,Electronics
498,ITEM0499,5.12,90.64,37.67,6.44,Home


In [6]:
#### some pre processing
static_feature_cols = ['item_height','item_width','item_length','item_weight','article_category']
for col in static_feature_cols:
    if col in df_ts.columns:
        df_ts = df_ts.drop(col, axis=1)

df_ts['timestamp'] = pd.to_datetime(df_ts['timestamp'])
bool_cols = df_ts.select_dtypes(include=['bool']).columns
for col in bool_cols:
    df_ts[col] = df_ts[col].astype(int)
df_ts.dtypes

item_id                       object
timestamp             datetime64[ns]
target                       float64
net_revenue_sum              float64
listing_price_mean           float64
net_unit_price               float64
promo_flag                     int64
holiday_flag                   int64
weekend_flag                   int64
holiday_ahead                  int64
promo_ahead                    int64
dtype: object

#### Initialize values and variables

In [7]:
known_covariates = ['listing_price_mean','net_unit_price','promo_flag','holiday_flag','weekend_flag','holiday_ahead','promo_ahead']
prediction_length = 90
start_date = '2020-01-01'
test_start_date="2023-11-03" 
test_end_date="2024-02-01" 
TARGET = 'target'
trial_name = 'autogluon_ensemble_trial_1'
model_path = f'/home/sagemaker-user/Homepro_AWS_competancy/models/{trial_name}'
TRAIN_TIME_LIMIT =12000

##### perform train-test split 

In [8]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
ts = TimeSeriesDataFrame.from_data_frame(df_ts)
ts.static_features = df_static.set_index("item_id")
ts = ts.sort_index()
print("converted to timeseries df")
filtered_ts = ts.slice_by_time(start_time=pd.Timestamp(start_date), end_time=pd.Timestamp(test_end_date))
train_data, test_data = filtered_ts.train_test_split(prediction_length)

2024-06-18 23:20:06.601124: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 23:20:06.649185: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


converted to timeseries df


##### min history has to be 2*prediction_length+1 days (prediction_length days is the test period)

In [9]:
min_date = pd.Timestamp(test_end_date) - pd.Timedelta(days=3*prediction_length+1)
filtered_test_data = test_data.slice_by_time(start_time=min_date, end_time=pd.Timestamp(test_end_date))
test_item_counts = filtered_test_data.groupby(level=0).size()
# Sort the counts in ascending order
sorted_counts = test_item_counts.sort_values()
test_filtered_items_to_remove = sorted_counts[sorted_counts < 3*prediction_length+1].index.tolist()
train_data = train_data[~train_data.index.get_level_values(0).isin(test_filtered_items_to_remove)]
test_data = test_data[~test_data.index.get_level_values(0).isin(test_filtered_items_to_remove)]

### Model selection 

In [13]:
predictor = TimeSeriesPredictor(
        prediction_length = prediction_length,
        path=f'{model_path}',
        target=TARGET,
        eval_metric='MAPE',#'MASE',#'RMSE',
        known_covariates_names=known_covariates,
        verbosity=4,
)
# ## Training AutoGluon Model
predictor.fit(
        train_data,
        presets='best_quality',
        time_limit=TRAIN_TIME_LIMIT,
        random_seed=12345,
        # hyperparameters={
        #     "DeepAR":{#}
        #         "batch_size":64,
        #         "num_batches_per_epoch": 22,
        #         "hidden_dim": 128,
        #         "max_epochs": 100, 
        #         "context_length": 120,
        #         "num_layers": 3,
        #     },
        # },
        # # hyperparameter_tune_kwargs=None
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: best_quality
Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': 'MAPE',
 'excluded_model_types': None,
 'freq': None,
 'hyperparameter_tune_kwargs': {'num_trials': 3,
                                'scheduler': 'local',
                                'searcher': 'auto'},
 'hyperparameters': 'best_quality',
 'num_val_windows': 1,
 'prediction_length': 90,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 12345,
 'refit_full': False,
 'target': 'target',
 'time_limit': 12000,
 'verbosity': 4}

Inferred data frequency: D
Provided training data set with 113462 rows, 97 items (item = single time series). Average time series length is 1169.7. 
Global seed set to 12345
Beginning AutoGluon training with TimeSeriesLearner Time limit = 11999.97405052185
AutoGluon will save models to /home/sagemaker-user/Homepro_AWS_competancy/models/autogluo

  0%|          | 0/3 [00:00<?, ?it/s]

	Hyperparameter tune run: DeepAR/T1
		-0.6924      = Validation score (MAPE)
		99.831  s    = Training runtime
		1.666   s    = Training runtime
	Hyperparameter tune run: DeepAR/T2
		-0.6592      = Validation score (MAPE)
		96.092  s    = Training runtime
		1.901   s    = Training runtime
	Hyperparameter tune run: DeepAR/T3
		-0.6643      = Validation score (MAPE)
		114.307 s    = Training runtime
		4.401   s    = Training runtime
	Trained 3 models while tuning DeepAR.
	-0.6592       = Validation score (-MAPE)
	310.33  s     = Total tuning time
	Best hyperparameter configuration: {'num_layers': 2, 'hidden_size': 59}
Hyperparameter tuning model: TemporalFusionTransformer. Tuning model for up to 1200.00s of the 11999.97s remaining.
	Window 0
GluonTS logging is turned on during training. Note that losses reported by GluonTS may not correspond to those specified via `eval_metric`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0

#### we can see that DeepAR is performing the best with the HP configuration suggested
	Hyperparameter tune run: DeepAR/T1
		-0.6924      = Validation score (MAPE)
		99.831  s    = Training runtime
		1.666   s    = Training runtime
	Hyperparameter tune run: DeepAR/T2
		-0.6592      = Validation score (MAPE)
		96.092  s    = Training runtime
		1.901   s    = Training runtime
	Hyperparameter tune run: DeepAR/T3
		-0.6643      = Validation score (MAPE)
		114.307 s    = Training runtime
		4.401   s    = Training runtime
	Trained 3 models while tuning DeepAR.
	-0.6592       = Validation score (-MAPE)
	310.33  s     = Total tuning time
	Best hyperparameter configuration: {'num_layers': 2, 'hidden_size': 59}